`CMPE-255. Fall 2023`

# Sopa de Letras
Also known as "Word Search Puzzle", this game consists of the letters of words placed in a grid. The objective of this puzzle is to find all the words hidden inside the grid.

In [1]:
# Libraries - ONLY THIS LIBRARIES ARE ALLOWED FOR THE HOMEWORK
import numpy as np
import pandas as pd
import random
import sys
# Do not add any other library

In [2]:
# Soup size
DIMENSIONS = (8, 8)
# Word to insert in the soup
WORD = "MINING"
#Words to insert in the thick soup
WORDS = ["DATA", "MINING", "SJSU"]
# English alphabet
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [3]:
def create_random_broth(dimensions, alphabet):
  '''Create a random soup of letters.
  Return the soup as a n-dimensional array

  dimensions: Tuple with dimensions. E.g., (8, 8, 8) for a 3-D 8x8x8 array
  alphabet  : String with alphabet to use for letters in the broth
  '''

  # Create a multi-dimentional array of random numbers
  a = np.random.randint(0, len(alphabet)-1, dimensions)
  # Swap numbers in the array with characters from the alphabet
  my_dict = dict(zip(range(len(alphabet)), list(alphabet)))
  broth = np.vectorize(my_dict.get)(a)

  return broth

In [4]:
def add_word_to_broth(broth, word):
  '''Add "word" at random locations.
       Return the updated soup'''

  msoup = broth.copy()
  dimensions = broth.shape
  word_length = len(word)
  # Words are inserted as "lowercase", so it's easy to check
  not_suitable = True
  # Check if the randomly-selected insertion point is feasible
  max_attempts = 100
  while (not_suitable):
    max_attempts -= 1
    if (max_attempts == 0):
      print("ERROR - Unable to add word, after 100 attempts")
      return []
    # Pick a random start point
    start_point = [0] * len(dimensions)
    for i in range(len(dimensions)):
      start_point[i] = np.random.randint(0, dimensions[i])
    end_point = start_point.copy()
    # Pick a random dimension to place the word
    dim = np.random.randint(0, len(dimensions))
    # Calculate the end point of the word along the selected dimension
    end_point[dim] = start_point[dim] + len(word) - 1
    # Calculate the indexes in the array that the word would use when placed
    idx_increment = [0] * len(dimensions)
    idx_increment[dim] = 1
    indexes = [[start_point[j] + i*idx_increment[j] for j in range(len(start_point))] for i in range(len(word))]

    # Check if the word fits in the selected dimension
    if end_point[dim] >= dimensions[dim]:
      continue
    # Check if there is already a word (inserted words are always lowercase)
    c = [msoup[tuple(index)] for index in indexes]
    s = ''.join(c)
    if not s.isupper():
      continue

    # Add word
    for i in range(len(indexes)):
      msoup[tuple(indexes[i])] = word[i].lower()

    # Done with word
    not_suitable = False

  return np.char.upper(msoup)

In [5]:
def create_soup (dimensions, word, alphabet):
    '''Create a n-dimensional matrix of random letters (broth),
       then add words to it to make a soup.
       Return the soup'''

    broth = create_random_broth(dimensions, alphabet)
    soup  = add_word_to_broth(broth, word)
    return soup

In [6]:
def serve_soup(soup_plate, locations):
  '''Return highlighted DataFrame ready for display.
  rows and columns are subsets of the dataframe. For instance:
  rows = [2], cols = slice(1,6) - Highlight cells in row 2, from column 1 to 5
  '''
  styler = soup_plate.style
  styler.set_properties(**{'text-align': 'center'})
  for (rows, cols) in locations:
    styler.set_properties(subset=(rows, cols), **{'background': 'yellow'})

  return styler


In [7]:
def find_word_in_plate(word, soup_plate):
  '''Find a word in a 2-dimensional Pandas DataFrame'''

  result = []
  for icol in soup_plate.columns:
    for irow, row in soup_plate.iterrows():
      r = is_word_at_location(word, irow, icol, soup_plate)
      if r == 'h':
        result.append(([irow], slice(icol, icol+len(word)-1)))
      if r == 'v':
        result.append((slice(irow, irow+len(word)-1), [icol]))

  return result


def is_word_at_location(word, irow, icol, soup_plate):
  '''Examine a DataFrame at location (row, col) and
  determine if there is a "word"'''

  word_size = len(word)

  h_size = len(soup_plate.columns) - icol
  v_size = len(soup_plate.index) - irow

  # Check Horizontal
  if h_size >= word_size:
    res = True
    for i in range(len(word)):
      if soup_plate.iloc[irow, icol+i].upper() != word[i].upper(): res = False
    if res: return 'h'

  # Check Vertical
  if v_size >= word_size:
    res = True
    for i in range(len(word)):
      if soup_plate.iloc[irow+i, icol].upper() != word[i].upper(): res = False
    if res: return 'v'

  return ''

In [8]:

# Example 1 - Create a 8x8 puzzle, and insert the word "MINING".
# Find the word and display
%%time
s = create_soup((8,8), 'MINING', ALPHABET)
sl = slice(0, 8)
spoon = (sl, sl)
plate = pd.DataFrame(s[spoon])
words = find_word_in_plate("MINING", plate)
print(words)
table = serve_soup(plate, words)
table

[([7], slice(2, 7, None))]
CPU times: user 79.2 ms, sys: 2.05 ms, total: 81.3 ms
Wall time: 96.4 ms


,0,1,2,3,4,5,6,7
0,W,F,I,L,Y,D,V,B
1,O,I,H,W,T,W,P,U
2,U,M,O,S,D,E,X,G
3,G,E,W,C,M,K,M,K
4,R,X,E,E,J,A,L,X
5,N,Y,A,B,U,D,S,G
6,W,U,S,H,A,L,J,P
7,R,H,M,I,N,I,N,G


In [9]:
# Example 2 - Create a 8 x 8 x 4 puzzle, and insert the word "MINING"
# Create a plate (2-d dataframe), and check if the word is there
%%time
s = create_soup((8,8, 4), 'MINING', ALPHABET)
sl = slice(0, 8)
spoon = (sl, sl, 0)
plate = pd.DataFrame(s[spoon])
words = find_word_in_plate("MINING", plate)
print(words)
table = serve_soup(plate, words)
table

[]
CPU times: user 11.7 ms, sys: 890 µs, total: 12.6 ms
Wall time: 13.4 ms


,0,1,2,3,4,5,6,7
0,G,P,F,E,J,B,H,B
1,Q,J,L,E,A,I,O,F
2,V,K,H,T,Y,E,H,G
3,M,S,S,F,Y,T,W,R
4,T,P,B,I,H,A,T,N
5,D,T,S,B,Q,G,Q,V
6,T,W,V,L,V,N,R,A
7,O,B,I,K,F,T,N,E


# Homework Solution

## Deliverable 1
Setup Google Colab and run the provided notebook and example.
Add support for inserting multiple words in a puzzle. In order to do that, create two new functions:

```
add_words_to_broth(broth, words)
create_thick_soup(dimensions, words, alphabet)
```
The first function (add_words_to_broth) adds multiple words. Its argument "words" is a list of words in the form "["WORD1", "WORD2", ...].
The second function (create_thick_soup) uses "add_words_to_broth" to create a puzzle with multiple words inserted.

Create a 2-dimensional 16x16 soup with the following words added "DATA", "MINING", "SJSU", and display the results.

In [10]:
def add_words_to_broth(broth, words):
    '''Add multiple words to the soup'''
    msoup = broth.copy()
    for word in words:
        msoup = add_word_to_broth(msoup, word)
    return msoup

In [11]:
def create_thick_soup (dimensions, words, alphabet):
    '''Create a 16 x 16 puzzle with multiple words inserted'''
    broth = create_random_broth(dimensions, alphabet)
    soup  = add_words_to_broth(broth, words)
    return soup

In [12]:
%%time
dimensions = (16, 16)
s = create_thick_soup(dimensions, WORDS, ALPHABET)
spoon = slice(0, 16)
locations = []
plate = pd.DataFrame(s[spoon])
for word in WORDS:
  locations += find_word_in_plate(word, plate)
table = serve_soup(plate, locations)
table

CPU times: user 166 ms, sys: 169 µs, total: 167 ms
Wall time: 167 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,I,K,U,J,F,B,J,U,I,M,N,M,I,P,V,Y
1,E,J,C,P,W,A,F,S,X,S,Q,J,S,H,H,P
2,V,R,F,I,Y,U,G,J,Q,V,P,P,O,V,U,W
3,B,J,Y,Y,S,K,J,S,W,W,C,S,A,N,G,B
4,R,Q,N,C,B,I,Y,U,X,R,H,C,A,T,L,C
5,J,B,E,M,X,O,R,K,W,R,Y,B,R,H,K,H
6,A,G,R,L,U,Q,G,A,O,D,F,M,O,F,O,S
7,M,F,A,F,C,S,X,J,W,O,T,F,D,J,P,G
8,Y,R,P,V,D,E,N,L,U,L,J,B,C,K,U,U
9,N,I,K,E,C,I,H,N,K,A,F,V,U,Y,P,K


## Deliverable 2
The function that finds words in the puzzle is a very slow function with absolutely no optimization. Create a 2-dimensional soup of size 512 x 512. Insert the word "MINING" and measure the time to find the word using the provided function (find_word_in_plate).

Create a new optimized function to find words:
```
find_word_in_plate_faster(word, soup_plate)
```
This new function should include at least one optimization that makes finding the word faster.

Describe the optimization and how it helps finding words faster.
Run the find_word_in_plate_faster function in the 512x512 puzzle above and report the new time.

In [13]:
def find_word_in_plate_faster(word, soup_plate):
  '''Find a word faster in a 2-dimensional Pandas DataFrame'''
  result = []
  word_start = word[0]

  for irow, row in soup_plate.iterrows():
    if word_start in row.values:
      for icol, cell in row.items():
        if cell == word_start:
          r = is_word_at_location(word, irow, icol, soup_plate)
          if r == "h":
            result.append(([irow], slice(icol, icol + len(word)-1)))
          elif r == 'v':
            result.append((slice(irow, irow + len(word)-1), [icol]))

  return result

**The Optimization involved in the method find_word_in_plate_faster(word, soup_plate) is the reduction of unnecessary iterations and checks. This Optimization is achieved by checking the presence of the first letter of the target word in soup_plate before trying to find the word. In this way, it is possible to skip rows and columns that do not contain the first letter, thereby reducing the number of iterations.**



In [15]:
dimensions = (512, 512)
s = create_soup(dimensions, WORD, ALPHABET)
sl = slice(0, 512)
spoon = (sl, sl)
plate = pd.DataFrame(s[spoon])

# To Measure the time taken for the original function
start_time = np.datetime64('now')
words = find_word_in_plate(WORD, plate)
end_time = np.datetime64('now')
elapsed_time_original = end_time - start_time

# To Measure the time taken for the optimized function
start_time = np.datetime64('now')
words_faster = find_word_in_plate_faster(WORD, plate)
end_time = np.datetime64('now')
elapsed_time_optimized = end_time - start_time

print("Word searched in both the functions is:", WORD)
print("Time taken by original function:", elapsed_time_original)
print("Time taken by optimized function:", elapsed_time_optimized)

Word searched in both the functions is: MINING
Time taken by original function: 86 seconds
Time taken by optimized function: 4 seconds
